In [12]:
import pandas as pd
from scipy.spatial import distance
from sklearn.cluster import AgglomerativeClustering
import numpy as np

In [2]:
df1=pd.read_csv('D:/Data/Dacon공간정보/KRI-DAC_Jeju_data5.txt')
df2=pd.read_csv('D:/Data/Dacon공간정보/KRI-DAC_Jeju_data6.txt')
df3=pd.read_csv('D:/Data/Dacon공간정보/KRI-DAC_Jeju_data7.txt')
df4=pd.read_csv('D:/Data/Dacon공간정보/KRI-DAC_Jeju_data8.txt')

'''외부 데이터'''
#https://www.data.go.kr/data/15004770/fileData.do
place_df= pd.read_csv('D:/Data/Dacon공간정보/제주특별자치도_제주도장소(POI)데이터_20151231.csv',encoding='cp949')

In [3]:
for df in [df1,df2,df3,df4]:
    print(df.columns,df.shape)
    
df3.drop(['X','Y'],axis=1,inplace=True)
df=pd.concat([df1,df2,df3,df4])

df.drop(['OBJECTID','Field1'],axis=1,inplace=True)
df.columns = ['년월','지역대분류','지역중분류','소상공인구분','업종명','시간대','총사용금액','재난지원금 사용금액','총 이용건수','총 재난지원금 이용건수','X좌표','Y좌표']

df.head()

Index(['OBJECTID', 'Field1', 'YM', 'SIDO', 'SIGUNGU', 'FranClass', 'Type',
       'Time', 'TotalSpent', 'DisSpent', 'NumofSpent', 'NumofDisSpent',
       'POINT_X', 'POINT_Y'],
      dtype='object') (273183, 14)
Index(['OBJECTID', 'Field1', 'YM', 'SIDO', 'SIGUNGU', 'FranClass', 'Type',
       'Time', 'TotalSpent', 'DisSpent', 'NumofSpent', 'NumofDisSpent',
       'POINT_X', 'POINT_Y'],
      dtype='object') (281896, 14)
Index(['OBJECTID', 'Field1', 'YM', 'SIDO', 'SIGUNGU', 'X', 'Y', 'FranClass',
       'Type', 'Time', 'TotalSpent', 'DisSpent', 'NumofSpent', 'NumofDisSpent',
       'POINT_X', 'POINT_Y'],
      dtype='object') (284265, 16)
Index(['OBJECTID', 'Field1', 'YM', 'SIDO', 'SIGUNGU', 'FranClass', 'Type',
       'Time', 'TotalSpent', 'DisSpent', 'NumofSpent', 'NumofDisSpent',
       'POINT_X', 'POINT_Y'],
      dtype='object') (280085, 14)


,년월,지역대분류,지역중분류,소상공인구분,업종명,시간대,총사용금액,재난지원금 사용금액,총 이용건수,총 재난지원금 이용건수,X좌표,Y좌표
0,202005,제주특별자치도,제주시,영세,일반한식,00시,363000,66500,10,2,877005.9834,1.479766e+06
1,202005,제주특별자치도,제주시,영세,단란주점,00시,1180000,0,3,0,877005.7447,1.479816e+06
2,202005,제주특별자치도,제주시,중소1,편의점,00시,157670,6850,20,2,877056.6756,1.479616e+06
3,202005,제주특별자치도,제주시,영세,편의점,00시,46600,0,2,0,877055.9593,1.479766e+06
4,202005,제주특별자치도,제주시,영세,주점,00시,66000,0,2,0,877055.4817,1.479866e+06


# JensenShannon Divergence

In [13]:
def JensenShannon_similarity(df,time_list,st_list,cal_col):
      '''
      [함수 사용법]
        X축에 들어갈 변수를 input_list 인덱싱 0번 자리(맨앞)에 둘 것
        그 뒤로 기준을 잡고 싶은 변수들을 차례로 넣어 주면 됩니다.
        from scipy.spatial import distance 꼭 할것!
      '''


      make_amt= df.groupby(time_list+st_list)[cal_col].sum().reset_index()

      '''pivot 만들기'''
      amt_sum = make_amt.groupby(st_list)[cal_col].sum().reset_index(name='cal_col_sum')
      amt_sum = amt_sum[amt_sum['cal_col_sum'] != 0] #JSD를 구할때 0이 있으면 문제가 생김
      make_amt = make_amt.merge(amt_sum,on=st_list,how='left')
      make_amt['cal_col_prob'] = make_amt[cal_col]/make_amt['cal_col_sum']

      # pivot_table을 활용하여 long to wide 작업 실시
      amt_pivot= pd.pivot_table(make_amt,
                                index=st_list,
                                columns=time_list,
                                values='cal_col_prob').reset_index()
      amt_pivot.fillna(0,inplace=True)

      #피벗된 연도별 amt를 사용해 고객간의 유사도를 구해보자.
      amt_array=np.array(amt_pivot.drop(st_list,axis=1))

      js_similarity =[]
      for row_r in range(amt_array.shape[0]):       
        dt_list=[]
        for row_c in range(amt_array.shape[0]):
            #dt= js_divergence(amt_pivot.iloc[row_r,3:].values,amt_pivot.iloc[row_c,3:].values)
            '''
            js_divergence(만든 함수)로 하면 0값을 계산을 못함. 
            sqrt(js_divergence)인 distance.jensenshannon 함수 사용
            '''
            dt = distance.jensenshannon(amt_array[row_r,:],amt_array[row_c,:],2.0)
            dt_list.append(dt)
        js_similarity.append(dt_list)

      js_frame =pd.DataFrame(np.array(js_similarity))
      js_similarity = pd.concat([amt_pivot,js_frame],axis=1)

      return amt_sum[st_list],js_similarity

## 소상공인구분

In [21]:
st_series,c_df=JensenShannon_similarity(df,['시간대'],['소상공인구분'],'총사용금액')

In [22]:
st_series

,소상공인구분
0,영세
1,일반
2,중소
3,중소1
4,중소2


In [26]:
c_df=c_df.iloc[:,-c_df.shape[0]:]

In [36]:
c_df.index = st_series['소상공인구분'].values
c_df.columns = st_series['소상공인구분'].values

In [37]:
c_df

,영세,일반,중소,중소1,중소2
영세,0.000000,0.212182,0.056863,0.089532,0.163537
일반,0.212182,0.000000,0.186959,0.147134,0.064574
중소,0.056863,0.186959,0.000000,0.048240,0.138373
중소1,0.089532,0.147134,0.048240,0.000000,0.098549
중소2,0.163537,0.064574,0.138373,0.098549,0.000000


## 업종구분

In [39]:
st_series,c_df=JensenShannon_similarity(df,['시간대'],['업종명'],'총사용금액')
c_df=c_df.iloc[:,-c_df.shape[0]:]
c_df.index = st_series['업종명'].values
c_df.columns = st_series['업종명'].values

In [40]:
c_df

,1급호텔,2급호텔,CATV,DVD음반테이프판매,LPG,가례서비스,가방,가전제품,가정용품수리,건강식품(회원제형태),...,한약방,한의원,항공사,헬스크럽,홍삼제품,화랑,화물운송,화방표구점,화원,화장품
1급호텔,0.000000,0.347502,0.740276,0.542423,0.328093,0.429539,0.481436,0.446365,0.501664,0.846882,...,0.496347,0.535399,0.403414,0.302763,0.454225,0.407569,0.351685,0.535921,0.413057,0.348853
2급호텔,0.347502,0.000000,0.726688,0.469346,0.243768,0.401512,0.377777,0.279271,0.328304,0.765386,...,0.371331,0.341376,0.332178,0.232293,0.316097,0.358008,0.366016,0.345934,0.259293,0.146357
CATV,0.740276,0.726688,0.000000,0.801295,0.656681,0.806082,0.774822,0.730620,0.636925,0.888888,...,0.643875,0.685082,0.700429,0.736148,0.728531,0.781503,0.692481,0.683775,0.674679,0.697504
DVD음반테이프판매,0.542423,0.469346,0.801295,0.000000,0.440184,0.547490,0.341076,0.343123,0.425477,0.562225,...,0.419095,0.437201,0.607354,0.432889,0.280118,0.405072,0.551729,0.505409,0.370192,0.398264
LPG,0.328093,0.243768,0.656681,0.440184,0.000000,0.438947,0.391924,0.281064,0.263213,0.743658,...,0.303209,0.304547,0.374379,0.263340,0.277854,0.389631,0.281638,0.336942,0.192202,0.199249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
화랑,0.407569,0.358008,0.781503,0.405072,0.389631,0.439561,0.362316,0.315465,0.429284,0.762310,...,0.410636,0.477080,0.460936,0.301728,0.319211,0.000000,0.484087,0.473944,0.352598,0.318279
화물운송,0.351685,0.366016,0.692481,0.551729,0.281638,0.513736,0.441732,0.405731,0.407905,0.819565,...,0.390270,0.440690,0.448685,0.420111,0.413892,0.484087,0.000000,0.436097,0.350489,0.366907
화방표구점,0.535921,0.345934,0.683775,0.505409,0.336942,0.584607,0.398904,0.263530,0.182872,0.746933,...,0.245805,0.182457,0.451377,0.460937,0.316323,0.473944,0.436097,0.000000,0.263636,0.324951
화원,0.413057,0.259293,0.674679,0.370192,0.192202,0.492550,0.281187,0.155025,0.156392,0.729007,...,0.214250,0.239750,0.418707,0.317393,0.172672,0.352598,0.350489,0.263636,0.000000,0.184428
